In [1]:
import os
import random
import gc
import multiprocessing
import warnings
from tqdm.auto import tqdm

import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import default_collate

from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import DataCollatorWithPadding

from utils import get_title, preprocess_text_field, MeanPooling, Attention

def seed_everything(seed=42, deterministic=False):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = deterministic
    torch.backends.cudnn.benchmark = False

#### Основные настройки: seed, модель, рабочий каталог, warnings.

In [2]:
SEED = 42
WORKDIR = '/home/maksim/KazanExpress/2/'
IMAGES_FOLDER = os.path.join(WORKDIR, 'row_data/images/train/')
warnings.filterwarnings("ignore")
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = 'true'
seed_everything(SEED)

%env TOKENIZERS_PARALLELISM=false

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device: ", device)
print('CPU cores: ', multiprocessing.cpu_count())

# =========================================================================================
# Configurations
# =========================================================================================
class CFG:
    num_workers = multiprocessing.cpu_count()
    clip_embeddings = np.load(os.path.join(WORKDIR, 'embeddings.np.npy'))
    clip_cut_emb = 32
    bert_model = 'cointegrated/rubert-tiny2' 
    bert_tokenizer = AutoTokenizer.from_pretrained('cointegrated/rubert-tiny2')
    bert_cut_emb = None
    state_dict = torch.load(os.path.join(WORKDIR, 'checkpoint_clip_bert_kaggle.pt'))
    max_length = 256


env: TOKENIZERS_PARALLELISM=false
Device:  cuda
CPU cores:  6


#### Преобразование входных данных

In [3]:
# Read from parquet
data_full = pd.read_parquet(os.path.join(WORKDIR, 'row_data/train.parquet'))
# Drop unnecessary columns
data_full.drop(columns=['shop_id', 'rating'], inplace=True)
# Convert text fields
data_full['title'] = data_full.text_fields.apply(get_title)
data_full.text_fields = data_full.text_fields.apply(preprocess_text_field)
# Convert "Sale"
data_full['sale'] = data_full['sale'].apply(lambda x: "Распродажа!" if x else "")  
data_full.fillna(value='', inplace=True)
# Concatenate to one string
data_full = data_full.assign(Document=[str(y) + ': ' + str(x) + '. ' + str(z) + '. ' + str(s) + '. ' \
                                       for x, y, z, s in zip(data_full['title'], data_full['shop_title'],
                                                           data_full['text_fields'], data_full['sale'])])

data_full = data_full.drop(columns=['text_fields', 'shop_title', 'sale', 'title']).reset_index(drop=True)
# Drop too rare values
drop_ids = set(data_full.category_id.value_counts()[data_full.category_id.value_counts() < 2].index)
data_full = data_full[~data_full['category_id'].isin(drop_ids)]
# Trait/test split
if CFG.clip_embeddings is not None:
    data, data_valid, clip_embeddings, clip_embeddings_valid = train_test_split(data_full, CFG.clip_embeddings, 
                                                                    test_size=0.2, random_state=SEED, 
                                                                    shuffle=True, stratify=data_full.category_id)
else:
    data, data_valid_stack = train_test_split(data_full, test_size=0.2, random_state=SEED, 
                                        shuffle=True, stratify=data_full.category_id)
    
data.reset_index(drop=True, inplace=True)
data_valid.reset_index(drop=True, inplace=True)
# Fix class umbers 
cls2id = data_full.category_id.unique()
id2cls = {k : v for v, k in enumerate(cls2id)}
# Make categories dictionary
id2category = {k:v[15:] for k, v in zip(data_full.category_id.tolist(), data_full.category_name.tolist())}
# del data_full

#### Классы датасета и модели. 
Модель и датасет позволяют загружать как модель CLIP (собственную или с huggingface), так и готовые (ранее сгенерированные и сохранунные в numpy array) эмбеддинги CLIP.

In [4]:
# =========================================================================================
# Dataset
# =========================================================================================
class stacked_dataset(Dataset):
    def __init__(self, cfg, documents:list, targets: list, 
                 id2cls: dict, images_folder: str, 
                 product_ids:list, clip_embeddings=None):
        
        if clip_embeddings is not None:
            self.use_precalculated_clip_embs = True
            self.clip_embeddings = clip_embeddings
        else:
            self.use_precalculated_clip_embs = False
        
        self.cfg = cfg
        self.data = documents
        self.targets = targets
        self.id2cls = id2cls
        self.images_folder = images_folder
        self.product_ids = product_ids
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, item):
        if self.use_precalculated_clip_embs:
            image_inputs = self.clip_embeddings[item][None, :]
        else:
            image = Image.open(os.path.join(self.images_folder, str(self.product_ids[item]) + '.jpg'))
            image_inputs = self.cfg.clip_processor(
                    text=None,
                    images=image,
                    return_tensors='pt'
                )['pixel_values']
        text_inputs=self.cfg.bert_tokenizer(
                self.data[item], 
                return_tensors=None, 
                add_special_tokens=True, 
                max_length=self.cfg.max_length,
                truncation=True
            )
        return text_inputs, image_inputs, self.id2cls[self.targets[item]]

# =========================================================================================
# Classifier model
# =========================================================================================   
class STACKED_CLF(nn.Module):
    def __init__(self, cfg, n_classes):
        super().__init__()
        # Configurations, CLIP and BERT models loading
        self.cfg = cfg
        if cfg.clip_embeddings is None:
            self.clip_config = AutoConfig.from_pretrained(cfg.model)
            self.clip_model = CLIPModel.from_pretrained(cfg.model)
        else: 
            self.use_precalculated_clip_embs = True
        self.bert_config = AutoConfig.from_pretrained(cfg.bert_model)
        self.bert_model = AutoModel.from_pretrained(cfg.bert_model, config = self.bert_config)
        self.bert_pool = MeanPooling()
        # CLIP embeddings from model or precalculated
        if cfg.bert_cut_emb is None:
            self.hidden_dim = self.bert_model.config.hidden_size + cfg.clip_cut_emb
        else:
            self.hidden_dim = cfg.bert_cut_emb + cfg.clip_cut_emb
        # Attentions
        self.attention_clip = Attention(self.hidden_dim-cfg.clip_cut_emb, cfg.clip_cut_emb)
        self.attention_bert = Attention(cfg.clip_cut_emb, self.hidden_dim-cfg.clip_cut_emb)
        # Classifier
        self.bn = nn.BatchNorm1d(self.hidden_dim)
        self.clf = nn.Linear(self.hidden_dim, n_classes)

    def forward(self, text_inputs, image_inputs):
        # Get BERT embeddings from text
        text_emb = self.bert_model(**text_inputs)
        text_emb = self.bert_pool(text_emb.last_hidden_state, text_inputs['attention_mask']) 
        # Get CLIP embeddings from pictures
        if self.use_precalculated_clip_embs:
            img_emb = image_inputs 
        else:
            img_emb = self.clip_model.get_image_features(image_inputs)
        # Cut embeddings
        text_emb = text_emb[:, :self.cfg.bert_cut_emb]
        img_emb = img_emb[:, :self.cfg.clip_cut_emb]
        # Apply attentions
        img_emb = self.attention_clip(text_emb, img_emb)
        text_emb = self.attention_bert(img_emb, text_emb)
        # Concatenate BERT and CLIP embeddings
        emb  = torch.cat([text_emb, img_emb], dim=1).float()
        # Classifier
        cls = self.clf(self.bn(emb))
        return cls

Функция collate_fn и даталоадер:

In [5]:
transformers_collator = DataCollatorWithPadding(tokenizer = CFG.bert_tokenizer, padding = 'longest')

def custom_collate(batch):
    texts_batch = []
    images_batch = []
    targets_batch = []
    for item in batch:
        texts_batch.append(item[0])
        images_batch.append(item[1][0])
        targets_batch.append(item[2])
    text_inputs = transformers_collator(texts_batch)
    return text_inputs, default_collate(images_batch), default_collate(targets_batch)

valid_loader = DataLoader(
    stacked_dataset(CFG, documents=data_valid.Document.tolist(), targets=data_valid.category_id.tolist(), 
                  id2cls=id2cls, images_folder=IMAGES_FOLDER, 
                  product_ids=data_valid.product_id.tolist(), clip_embeddings=clip_embeddings_valid), 
    batch_size = 256, 
    shuffle = False, 
    collate_fn = custom_collate,
    num_workers = CFG.num_workers, 
    pin_memory = True, 
    drop_last = False
)


Создание модели:

In [6]:
model = STACKED_CLF(CFG, len(cls2id)).to(device)
if CFG.state_dict is not None:
    model.load_state_dict(CFG.state_dict)
torch.cuda.empty_cache()
gc.collect()

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


17

In [7]:
def get_categories(model, test_loader):
    grun_truth = []
    predicted = []
    model.to(device)
    model.eval()
    with torch.no_grad():
        for batch in tqdm(test_loader, total = len(test_loader)):
            text_input = batch[0].to(device)
            image_input = batch[1].to(device)
            target = batch[2].to(device)
            output = model(text_input, image_input)
            grun_truth.append(target.cpu())
            predicted.append(output.argmax(dim=1).cpu())
    grun_truth = np.concatenate(grun_truth)
    predicted = np.concatenate(predicted)
    weighted_f1 = f1_score(grun_truth, predicted, average='weighted')
    print(f"F1={weighted_f1:.5f}")
    return predicted

Создаем датафрейм для просмотра ошибочно предсказанных категорий. Тот же сплит, что и в начале.

In [8]:
data_full = pd.read_parquet(os.path.join(WORKDIR, 'row_data/train.parquet'))
data_full = data_full[~data_full['category_id'].isin(drop_ids)]
_, data_watch, _, _ = train_test_split(data_full, data_full, test_size=0.2, random_state=SEED, 
                                        shuffle=True, stratify=data_full.category_id)

Делаем предсказание категорий:

In [9]:
predicted = get_categories(model, valid_loader)

  0%|          | 0/72 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 78.00 MiB (GPU 0; 5.93 GiB total capacity; 2.10 GiB already allocated; 81.00 MiB free; 2.12 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

Выбираем ошибочно предсказанные, приводим к читаемому виду:

In [ ]:
data_watch['predicted'] = [cls2id[x] for x in predicted] 
data_watch_wrong = data_watch[data_watch.category_id != data_watch.predicted]
data_watch_wrong['predicted_category'] = data_watch_wrong.predicted.apply(lambda x: id2category[x])
data_watch_wrong.category_name = data_watch_wrong.category_name.apply(lambda x: x[15:])
data_watch_wrong.drop(columns=['product_id', 'sale', 'sale', 'shop_id', 'shop_title', 'rating', 'category_id', 'predicted'], inplace=True)

#### Посмотрим на ошибки:

In [ ]:
for i, row in data_watch_wrong.head(20).iterrows():
    print(str(row[0])[:200])
    print('TRUE:', row[1])
    print('PRED:', row[2], '\n')

{"title": "Картина по номерам на холсте с подрамником 30х40 см \"Китайский дракон\"", "description": "<iframe class=\"ql-video\" src=\"https://www.youtube.com/embed/MBbjACIcU8s?showinfo=0&amp;rel=0&am
TRUE: Хобби и творчество->Создание картин, фоторамок, открыток->Картины по номерам->Другое
PRED: Хобби и творчество->Создание картин, фоторамок, открыток->Картины по номерам->Животные и птицы 

{"title": "Игровая клавиатура Thunder Wolf T20", "description": "<p>Игровая клавиатура Thunder Wolf T20 отлично подходит для работы и игр в темноте. <span style=\"color: rgb(51, 51, 51); background-co
TRUE: Электроника->Игровые приставки->Игровые контроллеры->Игровые клавиатуры
PRED: Электроника->Компьютерная техника->Аксессуары для компьютеров->Клавиатуры 

{"title": "Интерьерная табличка, декоративное панно", "description": "<p>Яркая деревянная картина \"Выходя из дома не забудь\", \"Правила этого дома\", \"Никогда не теряй терпения\", \"Благословение д
TRUE: Товары для дома->Декор и интерьер->Ка

#### Комментарий:
В тестовой выборке получилось 1988 (10% от исходного количества) товаров, на которых модель ошиблась, смотрим, что это за товары:

Пример, когда модель предсказывает правильно, но категория все равно не совпадает с разметкой (таких примеров много):
"{"title": "Картина по номерам на холсте с подрамником \"Двухцветный попугай\", 50x40 см"}
TRUE: Хобби и творчество->Создание картин, фоторамок, открыток->Картины по номерам->Другое
PRED: Хобби и творчество->Создание картин, фоторамок, открыток->Картины по номерам->Животные и птицы"

Пример, когда модель предсказывает неплохо, но немного не попадает в категорию (таких примеров большинство):
"{"title": "Настенный стеллаж для хранения специй и банки для специй"}
TRUE: Товары для дома->Товары для кухни->Порядок на кухне->Полки кухонные
PRED: Товары для дома->Товары для кухни->Хранение продуктов->Емкости для специй и мельницы"

Пример грубой ошибки модели (подобных ошибок среди первых 20 неверно предсказанных я нашел всего 4, т.е. 2% от исходного количества):
"{"title": "Наклейка знак \"Не мусорить\", 18х18 см"}
TRUE: Товары для дома->Декор и интерьер->Таблички, номера и крючки
PRED: Товары для дома->Хозяйственные товары->Мусорные ведра и баки"